In [1]:
from zeta_py.zeta_client.accounts.pricing import Pricing
from solana.rpc.async_api import AsyncClient
from solders.pubkey import Pubkey

In [2]:
Pricing

zeta_py.zeta_client.accounts.pricing.Pricing

In [5]:
conn = AsyncClient("https://api.mainnet-beta.solana.com")
p = await Pricing.fetch(conn, Pubkey.from_string("BbKFezrmKD83PeVh74958MzgFAue1pZptipSNLz5ccpk"))

In [12]:
p.__class__.__name__

'Pricing'

In [8]:
await p.fetch(conn, Pubkey.from_string("BbKFezrmKD83PeVh74958MzgFAue1pZptipSNLz5ccpk"))

Pricing(nonce=253, mark_prices=ListContainer([21376638, 26001680000, 1657779730, 5821087, 998837]), mark_prices_padding=ListContainer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), update_timestamps=ListContainer([1692383048, 1692383048, 1692383048, 1692383048, 1692383048]), update_timestamps_padding=ListContainer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), funding_deltas=[AnchorDecimal(flags=1507328, hi=3822981106, lo=3153921771, mid=4249199316), AnchorDecimal(flags=2148728832, hi=894704147, lo=483772813, mid=3611030112), AnchorDecimal(flags=1507328, hi=1790190011, lo=3873514900, mid=1912973221), AnchorDecimal(flags=2149056512, hi=3934200574, lo=2854070613, mid=1438644684), AnchorDecimal(flags=2149056512, hi=3173600808, lo=3966650445, mid=3324875027)], funding_deltas_padding=[AnchorDecimal(flags=0, hi=0, lo=0, mid=0), AnchorDecimal(flags=0, hi=0, lo=0, mid=0), AnchorDecimal(flags=0, hi=0, lo=0, mid=0), AnchorDecimal(flags=0, hi=0, lo=0, mid=0), Ancho